<a href="https://colab.research.google.com/github/SomdeepAcharyya/Recommender-Systems/blob/main/CREPE_MF_Towards_generating_scalable_personalized_recommendations_integrating_social_trust%2C_social_bias%2C_and_geo_spatial_clustering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Towards generating scalable personalized recommendations: integrating social trust, social bias, and geo-spatial clustering
# LR Divyaa, N Pervin
# Divyaa, L. R., & Pervin, N. (2019). Towards generating scalable personalized recommendations: integrating social trust, social bias, and geo-spatial clustering. Decision Support Systems, 122, 113066.

In [ ]:
import pandas as pd
import scipy, cmake
import numpy as np
import warnings
from sklearn import svm
import math
import scipy.spatial as sp
warnings.filterwarnings('ignore')

In [ ]:
# Amazon meta magazine Dataset
path = r'/content/drive/MyDrive/Per_CD_RS/Amazon_Meta_Magazine_Subscriptions.json'
with open(path, encoding="utf-8", errors='ignore') as infile:
  mz = pd.read_json(infile, lines=True)
mz = mz.rename(columns={"asin":"itemId"})

In [ ]:
# amazon review dataset magazines csv
path = r'/content/drive/MyDrive/Per_CD_RS/Amazon_Text_Magazine_Subscriptions.csv'
with open(path, encoding="utf-8", errors='ignore') as infile:
  rz = pd.read_csv(infile, nrows=12000)
rz = rz.rename(columns={"reviewerID":"userId", "asin":"itemId", "overall":"rating"})

In [ ]:
rz.shape

(12000, 13)

In [ ]:
mz.columns

Index(['category', 'tech1', 'description', 'fit', 'title', 'also_buy', 'tech2',
       'brand', 'feature', 'rank', 'also_view', 'details', 'main_cat',
       'similar_item', 'date', 'price', 'itemId', 'imageURL',
       'imageURLHighRes'],
      dtype='object')

In [ ]:
mz.shape

(3385, 19)

# rating matrix 3.1.1

In [ ]:
# masked rating matrix i 
iui = rz.pivot_table(index='userId',columns='itemId',values='rating')
iui = iui.fillna(0)
iui[iui>0] = 1 
print(iui.shape)
iui_arr = np.array(iui)

(11295, 66)


In [ ]:
# rating matrix wrt user u
ru = rz.pivot_table(index='userId',columns='itemId',values='rating')
ru = ru.fillna(0)
print(ru.shape)

(11295, 66)


In [ ]:
# user u mean rating rub
arr = np.array(ru)
rub = []
for i in range(len(arr)):
  sum = 0
  sum_d = 0
  for j in range(len(arr[i])):
    sum = sum + iui_arr[i][j] * arr[i][j]
    sum_d = sum_d + iui_arr[i][j]
  rub.append(sum/sum_d)

In [ ]:
# user u std dev rating rus
rus = []
arr = np.array(ru)
for i in range(len(rub)):
  sum = 0
  sum_d = 0
  for j in range(len(arr[i])):
    sum = sum + iui_arr[i][j] * (arr[i][j] - rub[i]) * (arr[i][j] - rub[i])
    sum_d = sum_d + iui_arr[i][j]
  rus.append(math.pow(sum/(sum_d-1), 0.5))
rus = [0 if x != x else x for x in rus]

In [ ]:
rus

In [ ]:
# rating matrix wrt item i
ri = rz.pivot_table(index='itemId',columns='userId',values='rating')
ri = ri.fillna(0)
print(ri.shape)

(66, 11295)


In [ ]:
# user u mean rating rub
arr = np.array(ri)
iui_arr_i = iui_arr.T
rib = []
for i in range(len(arr)):
  sum = 0
  sum_d = 0
  for j in range(len(arr[i])):
    sum = sum + iui_arr_i[i][j] * arr[i][j]
    sum_d = sum_d + iui_arr_i[i][j]
  rib.append(sum/sum_d)

In [ ]:
# item i std dev ris
ris = []
iui_arr_i = iui_arr.T
for i in range(len(rib)):
  sum = 0
  sum_d = 0
  for j in range(len(arr[i])):
    sum = sum + iui_arr_i[i][j] * (arr[i][j] - rib[i]) * (arr[i][j] - rib[i])
    sum_d = sum_d + iui_arr_i[i][j]
  ris.append(math.pow(sum/(sum_d-1), 0.5))
ris = [0 if x != x else x for x in ris]

In [ ]:
ris

# aspect factor 3.1.2

In [ ]:
a = mz[['itemId', 'category']]

In [ ]:
aspect = []
for i in mz['category']:
  for j in i:
    if j not in aspect:
      aspect.append(j)
aspect

In [ ]:
items = np.array(rz.itemId.value_counts().reset_index()['index'])
items

array(['B00005N7Q1', 'B00005N7SC', 'B00005N7OV', 'B00005N7TL',
       'B00005N7PN', 'B00005N7T5', 'B00005N7RD', 'B00005N7RA',
       'B00005N7TB', 'B00005N7SB', 'B00005N7QW', 'B00005N7SG',
       'B00005N7Q5', 'B00005N7SM', 'B00005N7QI', 'B00005N7SL',
       'B00005N7TG', 'B00005N7QN', 'B00005N7T3', 'B00005N7P0',
       'B00005N7QE', 'B00005N7OD', 'B00005N7QC', 'B00005N7PG',
       'B00005N7RT', 'B00005N7VQ', 'B00005N7NQ', 'B00005N7TH',
       'B00005N7R0', 'B00005N7QO', 'B00005N7T8', 'B00005N7SS',
       'B00005N7PS', 'B00005N7RJ', 'B00005N7O6', 'B00005N7R7',
       'B00005N7QL', 'B00005N7UC', 'B00005N7QH', 'B00005N7S8',
       'B00005N7S5', 'B00005N7OU', 'B00005N7SV', 'B00005N7XG',
       'B00005N7PI', 'B00005N7OC', 'B00005N7QS', 'B00005N7XI',
       'B00005N7OF', 'B00005N7TE', 'B00005N7TM', 'B00005N7RF',
       'B00005N7S4', 'B00005N7OA', 'B00005N7VL', 'B00005N7VO',
       'B00005N7OP', 'B00005N7TN', 'B00005N7R6', 'B00005N7UK',
       'B00005N7WD', 'B00005N7OJ', 'B00005N7SN', 'B0000

In [ ]:
item_att = []
for i in range(len(items)):
  x = items[i]
  t = list(mz[mz['itemId']==x]['category'])
  item_att.append(t)

In [ ]:
A = pd.DataFrame()
for x in aspect:
  arr = []
  for i in item_att:
    if x in i[0]:
      arr.append(1)
    else:
      arr.append(0)
  A[x] = arr 
aspect_arr = np.array(A)
A

,Magazine Subscriptions,Professional & Educational Journals,Professional & Trade,Humanities & Social Sciences,Economics & Economic Theory,"Arts, Music &amp; Photography",Music,Fashion &amp; Style,Women,"Sports, Recreation & Outdoors",...,Racket Sports,Spanish-Language,Interior Design,Country Life,Alternative & Holistic,Girls,Mathematics,Animals,Agricultural Sciences,Soccer
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
62,1,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
63,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
64,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
pu = []
for u in range(len(iui_arr)):
  t = 0
  c = 0
  for i in range(len(iui_arr[u])):
    t = t + aspect_arr[i] * iui_arr[u][i]
    if iui_arr[u][i] == 1: 
      c = c + 1
    c1 = t/c
  pu.append(c1)

In [ ]:
len(pu[0])

227

In [ ]:
len(pu[0])

227

In [ ]:
# cosine similarity user user
prefsim_u = 1 - sp.distance.cdist(pu, pu, 'cosine')

In [ ]:
prefsim_u

array([[1.        , 0.33333333, 1.        , ..., 0.28867513, 1.        ,
        0.33333333],
       [0.33333333, 1.        , 0.33333333, ..., 0.28867513, 0.33333333,
        1.        ],
       [1.        , 0.33333333, 1.        , ..., 0.28867513, 1.        ,
        0.33333333],
       ...,
       [0.28867513, 0.28867513, 0.28867513, ..., 1.        , 0.28867513,
        0.28867513],
       [1.        , 0.33333333, 1.        , ..., 0.28867513, 1.        ,
        0.33333333],
       [0.33333333, 1.        , 0.33333333, ..., 0.28867513, 0.33333333,
        1.        ]])

In [ ]:
theta = 0.33

In [ ]:
# user network pn construction
arr = prefsim_u.copy()
arr[arr>=theta] = 1 
arr[arr<theta] = 0
pn_u = arr
pn_u

array([[1., 1., 1., ..., 0., 1., 1.],
       [1., 1., 1., ..., 0., 1., 1.],
       [1., 1., 1., ..., 0., 1., 1.],
       ...,
       [0., 0., 0., ..., 1., 0., 0.],
       [1., 1., 1., ..., 0., 1., 1.],
       [1., 1., 1., ..., 0., 1., 1.]])

In [ ]:
# cosine similarity item item
pi = np.array(A)
prefsim_i = 1 - sp.distance.cdist(pi, pi, 'cosine')
prefsim_i

array([[1.        , 0.33333333, 0.66666667, ..., 0.28867513, 0.33333333,
        0.28867513],
       [0.33333333, 1.        , 0.33333333, ..., 0.28867513, 0.33333333,
        0.28867513],
       [0.66666667, 0.33333333, 1.        , ..., 0.28867513, 0.33333333,
        0.28867513],
       ...,
       [0.28867513, 0.28867513, 0.28867513, ..., 1.        , 0.28867513,
        0.25      ],
       [0.33333333, 0.33333333, 0.33333333, ..., 0.28867513, 1.        ,
        0.28867513],
       [0.28867513, 0.28867513, 0.28867513, ..., 0.25      , 0.28867513,
        1.        ]])

In [ ]:
pi.shape

(66, 227)

In [ ]:
# user network pn construction
arr = prefsim_i.copy()
arr[arr>=theta] = 1 
arr[arr<theta] = 0
pn_i = arr
pn_i

array([[1., 1., 1., ..., 0., 1., 0.],
       [1., 1., 1., ..., 0., 1., 0.],
       [1., 1., 1., ..., 0., 1., 0.],
       ...,
       [0., 0., 0., ..., 1., 0., 0.],
       [1., 1., 1., ..., 0., 1., 0.],
       [0., 0., 0., ..., 0., 0., 1.]])

# algorithm

In [ ]:
ru1 = ru.reset_index()
ru1

itemId,userId,B00005N7NQ,B00005N7O6,B00005N7OA,B00005N7OC,B00005N7OD,B00005N7OF,B00005N7OJ,B00005N7OP,B00005N7OU,...,B00005N7TN,B00005N7UC,B00005N7UK,B00005N7VL,B00005N7VO,B00005N7VQ,B00005N7WD,B00005N7WM,B00005N7XG,B00005N7XI
0,A0333047WGK24IZKLDP2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,A0534350D18UHJKPKZ1W,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,A1008WMRGO1W67,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,A100NRVPI4UOH,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,A100WO06OQR8BQ,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11290,AZXLG739A7D3N,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
11291,AZXT5KVWD0A3R,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
11292,AZY0YYSX5FYE5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
11293,AZZM0EGX7OIJH,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
ru2 = np.array(ru1).T
ru2

array([['A0333047WGK24IZKLDP2', 'A0534350D18UHJKPKZ1W', 'A1008WMRGO1W67',
        ..., 'AZY0YYSX5FYE5', 'AZZM0EGX7OIJH', 'AZZP6XXCKUBK1'],
       [0.0, 0.0, 0.0, ..., 0.0, 0.0, 0.0],
       [0.0, 0.0, 0.0, ..., 0.0, 0.0, 0.0],
       ...,
       [0.0, 0.0, 0.0, ..., 0.0, 0.0, 0.0],
       [0.0, 0.0, 0.0, ..., 0.0, 0.0, 0.0],
       [0.0, 0.0, 0.0, ..., 0.0, 0.0, 0.0]], dtype=object)

In [ ]:
print(np.array(pu).shape)
pu_df = pd.DataFrame(pu)
pu_df.columns = aspect
pu_df.insert(0, 'userId', ru.reset_index().userId)

(11295, 227)


In [ ]:
from numpy import where
from sklearn.datasets import make_classification
from matplotlib import pyplot
from sklearn.metrics import silhouette_score

In [ ]:
# K means line 3

params = [97]
from sklearn.cluster import KMeans
# define the model
for i in params:
  cluster = KMeans(n_clusters=i)
  cluster.fit_predict(pu)
  labels = cluster.labels_
  print(i, silhouette_score(pu, labels), len(set(cluster.labels_)))
pu_df['cluster'] = labels

97 0.9751172163457553 97


In [ ]:
# DBSCAN

from sklearn.cluster import DBSCAN
# define the model
eps = [0.3]
min_samples = [3]
for i in eps:
  for j in min_samples:
    cluster = DBSCAN(eps=i, min_samples=j)
    cluster.fit_predict(pu)
    labels = cluster.labels_
    print(i, j, silhouette_score(pu, labels), len(set(cluster.labels_)))

0.3 3 0.9734587858805932 87


In [ ]:
# initialisation line 4
c = 1
M = []
N = []
R = []

In [ ]:
for i in range(len(set(labels))):
  t = np.array(pu_df[pu_df['cluster']== i]['userId']) # t = set of users in cluster i
  temp = []
  for v in t:
    row = []                               # v = each user in cluster i
    t1 = np.array(ru1[ru1['userId']== v])[0]   # ratings profile of user v belonging to cluster i 
    t1 = t1[1:len(t1)]                         # all ratings given by user v belonging to cluster i
    for j in range(len(t1)):                
      if t1[j] != 0:
        row.append(j)                         # all items rated positive by user v in cluster i
    temp.append(row)                          # items positively voted by set of users in cluster i
  arr = []
  for x in temp:
    for y in x:
      arr.append(y)
  arr = set(arr)
  if len(t) <= 100:
    users = []
    items = []
    ratings = []
    M.append(t)
    N.append(arr)
    for x in t:
      t2 = np.array(rz[rz['userId']==x][['itemId', 'rating']])
      users.append(x)
      items.append(t2[0][0])
      ratings.append(t2[0][1])
    Rc = pd.DataFrame()
    Rc['users'] = users
    Rc['items'] = items
    Rc['ratings'] = ratings
    c = c+1
    print(1, "\n", Rc)
    if len(Rc) > 0:
      R.append(Rc)
  else:
    item_features = []
    for x in arr:
      item_features.append(pi[x])
    cluster = DBSCAN(eps=0.9, min_samples=3)
    cluster.fit_predict(item_features)
    item_labels = cluster.labels_
    item_df = pd.DataFrame()
    item_df['items'] = np.array(list(arr))
    item_df['cluster'] = item_labels
    #print(item_df)
    for x in range(item_df['cluster'].min(), item_df['cluster'].max()+1):
      is1 = np.array(item_df[item_df['cluster']==x]['items'])
      users = []
      ratings = []
      items = []
      Rc = pd.DataFrame()
      for y in is1:
        arr3 = []
        is2 = ru2[(1+y)]
        #print(set(is2))
        for z in range(len(is2)):
          if is2[z] > 0.0:
            users.append(ru2[0][z])
            ratings.append(ru2[y][z])
      for z in range(len(users)):
          arr3.append(ru1.columns[1+y])
      #print(y, "[", is1, "]", len(users), len(arr3))
      items.append(arr3)
      #print(len([item for sublist in items for item in items]))    
      Rc['users'] = users
      Rc['items'] = items[0]
      Rc['ratings'] = ratings
      Rc = Rc[Rc['ratings']!=0.0]
      print(0, "\n", Rc)
      c = c+1
      M.append(users)
      N.append(items[0])
      if len(Rc) > 0:
        R.append(Rc)
  c = c-1

0 
                users       items  ratings
26    A128IR7BENTWLD  B00005N7TG      3.0
294   A1QCCDV4RAIVWZ  B00005N7TG      4.0
499    A29PE7S3S4TCG  B00005N7TG      1.0
592   A2HSTO3CIZAP0R  B00005N7TG      4.0
911   A39U7CYRJGN71Y  B00005N7TG      5.0
920   A3B6OPMZQ9DIO4  B00005N7TG      1.0
949   A3E6Z0TSNAQAJC  B00005N7TG      3.0
1120   A4SJWJHYIOWS4  B00005N7TG      2.0
1142   A68BZ1VAST1HY  B00005N7TG      5.0
1179    A9PXUENDO7Q0  B00005N7TG      2.0
1256   AGFZX4ELJSQ34  B00005N7TG      5.0
1305   AKCPXU46KZIY7  B00005N7TG      5.0
0 
 Empty DataFrame
Columns: [users, items, ratings]
Index: []
0 
 Empty DataFrame
Columns: [users, items, ratings]
Index: []
0 
                users       items  ratings
1590  A1K38RVM04NNEA  B00005N7R0      5.0
1623  A1UIGZ72QCG3H0  B00005N7R0      3.0
1736  A2XKQWF7HHHJXL  B00005N7R0      1.0
1758  A37EOWPXH0H84F  B00005N7R0      4.0
2347  A1YKUBH4Q408KH  B00005N7R0      5.0
0 
 Empty DataFrame
Columns: [users, items, ratings]
Index: []
0 
  

In [ ]:
M

[array(['A10A4CSWZ3BG1B', 'A14B73GG8OFXKH', 'A1614FDWEL04ND',
        'A17CWHXO2VFPAG', 'A18RIOMAB7DJQ', 'A1AX83WZI5MYPW',
        'A1D13ZTM8MBU9U', 'A1DQ3KRA6AD4KI', 'A1G1PUUA01TMPG',
        'A1GDGLAGBHBHDJ', 'A1GHMV7TGBKP21', 'A1JIGHMGQZNXT5',
        'A1KLDZ7YX2I3VR', 'A1LE6ISJQOHPZZ', 'A1O5QUPWS6CIF',
        'A1OC37ECBJL7F6', 'A1PI4BZEUTUYDQ', 'A1RMGEDX52MPZE',
        'A1W5L6FQDMVXNB', 'A1WCRMOU2GH3FZ', 'A1YNNZEK11LSJS',
        'A21YNZJ6SFG36V', 'A22RPSVJQX2CM1', 'A25H4QYCYJ4WIV',
        'A2AHFSS5D5JAEQ', 'A2B97MU7UL1Z3X', 'A2BXG4TQHX9WBC',
        'A2BYV7S1QP2YIG', 'A2GDP28AAO1KBD', 'A2J53HV9ZEZNXI',
        'A2OZPX02ZIKPJN', 'A2PMHCYMU6KMGB', 'A2V3P1XE33NYC3',
        'A2V8VLTPY3W01E', 'A2W9NU19AI75O7', 'A2Y2M19U2VJJPL',
        'A306CR82G5XEH2', 'A34BFOXOM87TCY', 'A35GTM4KD8K31X',
        'A383PPSVXNB0NB', 'A3880MQASTJFLX', 'A39MAFFU87L9F',
        'A3BFA6DHBTEQPJ', 'A3EPN0GK0Z0854', 'A3F7T693BJ4UTS',
        'A3GVCT8J4UM2SF', 'A3H72XNDTSY8DB', 'A3JBZ9QWT8H5DD',
        'A3

In [ ]:
N

[{24},
 {53},
 {15, 38},
 {0, 40, 43, 50, 52, 53},
 {0},
 {13},
 {36, 45},
 {1},
 {57},
 {16, 26, 27, 30, 50},
 {8, 64},
 {24, 29, 33},
 {20},
 {20, 23, 26, 30, 53},
 {64},
 {25},
 {46},
 {55},
 {5, 59},
 {0, 36, 40, 45, 49, 54},
 {9, 14, 16, 17, 21, 26, 31, 48},
 {3},
 {37},
 {32},
 {10, 16, 17, 19, 21, 22, 27, 40},
 {9, 14, 16, 43},
 {59},
 {9, 16, 17, 23, 24, 40, 43, 50, 54},
 {17, 31, 39, 40},
 {9, 41},
 {1, 8, 12, 18, 19, 23, 40, 43, 47, 52, 64},
 {10, 37, 40, 54},
 {18, 35, 41, 49},
 {14, 16, 17, 21, 24, 29, 50},
 {26, 40, 48, 52, 54},
 {9, 12, 26, 30, 41, 48},
 {10, 12, 14, 16, 17, 21, 24, 25, 31, 57},
 {58},
 {56},
 {20, 23, 35, 42},
 {9, 30, 41, 45},
 {19, 26, 31, 40, 50},
 {16, 17, 21, 22, 27, 48, 50, 51, 53, 54},
 {12, 14, 15, 18, 21, 23, 24, 31, 33, 35, 47},
 {18, 26, 30, 33, 47, 52, 54},
 {4, 18, 23, 52},
 {4, 26, 30, 52, 54},
 {9, 16, 17, 20, 26, 27, 39, 42, 50},
 {1, 10, 16, 17, 18, 23, 27, 29, 47, 50},
 {30, 31, 48, 54},
 {9, 16, 17, 27, 41, 50},
 {44},
 {23, 30, 40, 43

In [ ]:
R

[               users       items  ratings
 26    A128IR7BENTWLD  B00005N7TG      3.0
 294   A1QCCDV4RAIVWZ  B00005N7TG      4.0
 499    A29PE7S3S4TCG  B00005N7TG      1.0
 592   A2HSTO3CIZAP0R  B00005N7TG      4.0
 911   A39U7CYRJGN71Y  B00005N7TG      5.0
 920   A3B6OPMZQ9DIO4  B00005N7TG      1.0
 949   A3E6Z0TSNAQAJC  B00005N7TG      3.0
 1120   A4SJWJHYIOWS4  B00005N7TG      2.0
 1142   A68BZ1VAST1HY  B00005N7TG      5.0
 1179    A9PXUENDO7Q0  B00005N7TG      2.0
 1256   AGFZX4ELJSQ34  B00005N7TG      5.0
 1305   AKCPXU46KZIY7  B00005N7TG      5.0,
                users       items  ratings
 1590  A1K38RVM04NNEA  B00005N7R0      5.0
 1623  A1UIGZ72QCG3H0  B00005N7R0      3.0
 1736  A2XKQWF7HHHJXL  B00005N7R0      1.0
 1758  A37EOWPXH0H84F  B00005N7R0      4.0
 2347  A1YKUBH4Q408KH  B00005N7R0      5.0,
              users       items  ratings
 472  A461VTLW9G9YB  B00005N7TL      5.0,
               users       items  ratings
 92   A1WTD80C8J0QE2  B00005N7T8      2.0
 239  A3HL0OGT

In [ ]:
print(len(M), len(N), len(R))

2379 2379 72
